In [32]:
# Cell Counting and Co-localization Analysis Pipeline
"""
This notebook is the **second step** of a two-part protocol for automated cell counting 
in fluorescence microscopy images.

## Overview
This script performs comprehensive cell counting and co-localization analysis on:
- **Green channel (GFP)**: Specific cell population marker
- **Purple channel (c-fos)**: Neuronal activation marker  
- **Red channel (NeuN)**: General neuronal marker

## Prerequisites
Before running this notebook, ensure you have:
1. Processed .czi files using `Split_images_to_train.ipynb`
2. Generated segmentation masks using Cellpose
3. Manually defined ROIs (regions of interest) saved as .zip files

## Analysis Features
- **Multi-channel cell detection**: Counts cells in each fluorescence channel
- **Co-localization analysis**: Identifies cells expressing multiple markers
- **ROI-based analysis**: Processes user-defined regions of interest
- **3D analysis**: Handles multiple Z-planes independently
- **Export results**: Generates CSV tables and ROI files for ImageJ

## Output
- Detailed CSV table with cell counts per ROI and Z-plane
- ROI files (.zip) containing coordinates of co-localized cells
- Statistical summary of marker expression patterns
"""

# Import required libraries
import re                                    # Regular expressions for file pattern matching
from pathlib import Path                     # Modern file path handling
import os                                    # Operating system interface
from aicspylibczi import CziFile            # Reading .czi microscopy files
import numpy as np                           # Numerical computing
import matplotlib.pyplot as plt             # Plotting (optional visualization)
from roifile import roiread, ImagejRoi      # ImageJ ROI file handling
from skimage import measure                  # Image analysis and measurements
from skimage.draw import polygon             # Drawing geometric shapes on images
from skimage.measure import label as sklabel, regionprops  # Object labeling and properties
from skimage.segmentation import find_boundaries           # Boundary detection
import tifffile                             # TIFF image file handling
import pandas as pd                         # Data analysis and CSV export
from zipfile import ZipFile                 # ZIP file operations
import xml.etree.ElementTree as ET          # XML parsing for metadata

# Set matplotlib backend for interactive plotting
%matplotlib qt

# Function 1: Extract Pixel Size from CZI Metadata
def get_pixel_size_from_czi(czi):
    """
    Extracts pixel size information from CZI file metadata.
    
    This function reads the spatial calibration from the CZI file, which is essential
    for accurate distance measurements and ROI coordinate conversion.
    
    Parameters:
    - czi: CziFile object with loaded metadata
    
    Returns:
    - px_size_x: Pixel size in X dimension (micrometers)
    - px_size_y: Pixel size in Y dimension (micrometers)
    
    Raises:
    - ValueError: If spatial scales cannot be found in the CZI metadata
    """
    root = czi.meta  # Metadata is already parsed as an XML Element

    # Search for Distance elements with Id="X" and "Y"
    # These contain the spatial calibration information
    px_size_x = None
    px_size_y = None

    for dist in root.findall(".//Distance"):
        axis = dist.attrib.get("Id")
        value_elem = dist.find("Value")
        if value_elem is not None:
            if axis == "X":
                px_size_x = float(value_elem.text)
            elif axis == "Y":
                px_size_y = float(value_elem.text)

    # Ensure both X and Y scales were found
    if px_size_x is None or px_size_y is None:
        raise ValueError("Spatial scales X and Y not found in CZI metadata")

    return px_size_x, px_size_y


# Function 2: Convert ROI to Binary Mask
def roi_to_mask(roi, shape, px_size_x, px_size_y):
    """
    Converts an ImageJ ROI to a binary mask for image analysis.
    
    This function handles coordinate conversion from ROI format to pixel coordinates,
    automatically detecting whether coordinates are in micrometers or pixels.
    
    Parameters:
    - roi: ImageJ ROI object containing polygon coordinates
    - shape: Target image shape (height, width)
    - px_size_x: Pixel size in X dimension (micrometers)
    - px_size_y: Pixel size in Y dimension (micrometers)
    
    Returns:
    - mask: Binary mask array (True inside ROI, False outside)
    """
    # Initialize empty mask
    mask = np.zeros(shape, dtype=bool)
    coords = roi.coordinates()

    # Check if ROI has valid coordinates
    if coords is None or len(coords) < 3:
        return mask

    # Auto-detect coordinate units: micrometers vs pixels
    # Heuristic: if average coordinate value < 100, likely in micrometers
    if np.mean(coords) < 100:  # Coordinates appear to be in micrometers
        x_px = (coords[:, 0] / px_size_x).astype(int)
        y_px = (coords[:, 1] / px_size_y).astype(int)
    else:  # Coordinates already in pixels
        x_px = coords[:, 0].astype(int)
        y_px = coords[:, 1].astype(int)

    # Ensure coordinates are within image bounds
    x_px = np.clip(x_px, 0, shape[1] - 1)
    y_px = np.clip(y_px, 0, shape[0] - 1)

    # Create filled polygon mask
    rr, cc = polygon(y_px, x_px, shape)
    mask[rr, cc] = True

    return mask

# Function 3: Count Co-localized Objects
def count_overlaps(source_labels, target_mask, threshold=0.8, return_mask=False):
    """
    Identifies overlaps between labeled objects and a target mask using IoU analysis.
    
    This function implements sophisticated overlap detection that:
    1. Uses expanded bounding boxes for better context
    2. Calculates multiple IoU (Intersection over Union) metrics
    3. Handles both direct overlaps and contained objects
    
    Parameters:
    - source_labels: Labeled image with individual objects (e.g., cells from channel 1)
    - target_mask: Binary mask of target objects (e.g., cells from channel 2)
    - threshold: IoU threshold for considering objects as overlapping (default: 0.8)
    - return_mask: Whether to return a mask of overlapping regions
    
    Returns:
    - found_labels: Set of source labels that meet overlap criteria
    - overlap_mask: Binary mask of overlapping regions (if return_mask=True)
    """

    found_labels = set()
    overlap_mask = np.zeros_like(source_labels, dtype=bool)
    target_labels = sklabel(target_mask)

    nrows, ncols = source_labels.shape

    # Analyze each object in the source image
    for region_src in regionprops(source_labels):
        # Create binary mask for current source object
        mask_src = np.zeros_like(source_labels, dtype=bool)
        coords = tuple(np.transpose(region_src.coords))
        mask_src[coords] = True
        area_src = region_src.area

        # Create expanded bounding box (2x size) centered on the cell
        minr, minc, maxr, maxc = region_src.bbox
        center_r = (minr + maxr) // 2
        center_c = (minc + maxc) // 2
        height = maxr - minr
        width = maxc - minc

        # Expand bounding box while staying within image bounds
        minr_exp = max(0, center_r - height)
        maxr_exp = min(nrows, center_r + height)
        minc_exp = max(0, center_c - width)
        maxc_exp = min(ncols, center_c + width)

        # Extract regions from expanded bounding box
        sub_src = mask_src[minr_exp:maxr_exp, minc_exp:maxc_exp]
        sub_target = target_mask[minr_exp:maxr_exp, minc_exp:maxc_exp]
        sub_target_labels = target_labels[minr_exp:maxr_exp, minc_exp:maxc_exp]

        # Calculate direct overlap metrics
        area_intersection = np.sum(sub_src & sub_target)
        area_target_total = np.sum(sub_target)

        # IoU metrics: intersection/source and intersection/target
        iou_src = area_intersection / area_src if area_src > 0 else 0
        iou_target = area_intersection / area_target_total if area_target_total > 0 else 0

        # Check if direct overlap meets threshold
        if iou_src >= threshold or iou_target >= threshold:
            found_labels.add(region_src.label)
            if return_mask:
                overlap_mask[minr_exp:maxr_exp, minc_exp:maxc_exp] |= sub_src & sub_target
            continue

        # Check for target objects contained within source cell
        intersecting_labels = np.unique(sub_target_labels[sub_src])
        intersecting_labels = intersecting_labels[intersecting_labels > 0]

        for target_label in intersecting_labels:
            mask_target = sub_target_labels == target_label
            area_target = np.sum(mask_target)
            area_intersection2 = np.sum(mask_target & sub_src)

            # Additional IoU metrics for contained objects
            iou_contained_target = area_intersection2 / area_target if area_target > 0 else 0
            iou_contained_src = area_intersection2 / np.sum(sub_src) if np.sum(sub_src) > 0 else 0

            if iou_contained_target >= threshold or iou_contained_src >= threshold:
                found_labels.add(region_src.label)
                if return_mask:
                    overlap_mask[minr_exp:maxr_exp, minc_exp:maxc_exp] |= mask_target & sub_src
                break

    return (found_labels, overlap_mask) if return_mask else found_labels


# Function 4: Save Mask as ROI ZIP File
def save_mask_as_zip(mask, zip_path, base_name='ROI'):
    """
    Converts a binary mask to ImageJ ROI format and saves as ZIP file.
    
    This function extracts connected components from a binary mask and converts
    each component to an ImageJ ROI, saving all ROIs in a single ZIP file.
    
    Parameters:
    - mask: Binary mask containing objects to convert to ROIs
    - zip_path: Output path for the ZIP file containing ROIs
    - base_name: Base name for individual ROI files (default: 'ROI')
    
    Output:
    - ZIP file containing individual .roi files for each object
    """
    # Step 1: Label connected components in the mask
    labels = sklabel(mask)
    props = regionprops(labels)

    # Step 2: Create and save ROIs
    with ZipFile(zip_path, mode='w') as zipf:
        for i, prop in enumerate(props):
            coords = prop.coords  # (N, 2) array with (y, x) coordinates
            y, x = coords[:, 0], coords[:, 1]
            
            # Convert to ImageJ coordinate format (x, y)
            coords_ij = np.column_stack((x, y))  # (N, 2) array with (x, y)
            roi = ImagejRoi.frompoints(coords_ij)
            roi.name = f'{base_name}_{i}'

            # Save ROI to ZIP file
            roi_bytes = roi.tobytes()
            zipf.writestr(f"{roi.name}.roi", roi_bytes)

    print(f"[INFO] {len(props)} ROIs saved to {zip_path}")

# Function 5: Read Channel Images for Specific Z-plane
def read_channels_z(output_folder, z_fixed):
    """
    Reads channel images for a specific Z-plane from Cellpose output folder.
    
    This function locates and loads the segmentation masks for all channels
    at a specified Z-plane, organizing them by fluorescence marker.
    
    Parameters:
    - output_folder: Path to folder containing Cellpose segmentation results
    - z_fixed: Z-plane number to process
    
    Returns:
    - Dictionary with channel data:
      - 'green': GFP channel segmentation mask
      - 'purple': c-fos channel segmentation mask  
      - 'red': NeuN channel segmentation mask
      - 'mask_path': Path to one of the mask files (for naming output files)
    - None if insufficient channels found
    """
    # Create regex pattern to match files for the specified Z-plane
    pattern = re.compile(fr'_C[0-3]_Z{z_fixed}\.')
    files = [f for f in output_folder.glob('*.tiff') if pattern.search(f.name)]

    # Check if all required channels are present
    if len(files) < 4:
        print(f"[WARNING] Missing channels for Z = {z_fixed}, skipping...")
        return None

    # Load channel images and organize by marker type
    return {
        'green': tifffile.imread(files[0]),    # GFP channel (C0)
        'purple': tifffile.imread(files[1]),   # c-fos channel (C1)  
        'red': tifffile.imread(files[3]),      # NeuN channel (C3)
        'mask_path': files[0]  # Reference path for output file naming
    }

# Function 6: Process Individual ROI
def process_roi(i, roi, shape, px_size_x, px_size_y, masks_green, masks_purple, masks_red, z_fixed, mask_path):
    roi_mask = roi_to_mask(roi, shape, px_size_x, px_size_y)
    roi_border = find_boundaries(roi_mask, mode='outer')

    # Identify boundary cells that should be excluded from analysis
    labels_to_exclude = set()
    for mask in [masks_green, masks_purple, masks_red]:
        boundary_labels = np.unique(mask[roi_border])
        labels_to_exclude.update(boundary_labels[boundary_labels > 0])

    # Create filtered copies of channel masks
    green_filtered = masks_green.copy()
    purple_filtered = masks_purple.copy()
    red_filtered = masks_red.copy()

    # Remove boundary cells from all channels
    for lbl in labels_to_exclude:
        green_filtered[masks_green == lbl] = 0
        purple_filtered[masks_purple == lbl] = 0
        red_filtered[masks_red == lbl] = 0

    green_bin = (green_filtered > 0) & roi_mask
    purple_bin = (purple_filtered > 0) & roi_mask
    red_bin = (red_filtered > 0) & roi_mask

    green_labels = measure.label(green_bin)
    purple_labels = measure.label(purple_bin)
    red_labels = measure.label(red_bin)

    count_green = len(np.unique(green_labels[green_labels > 0]))
    count_purple = len(np.unique(purple_labels[purple_labels > 0]))
    count_red = len(np.unique(red_labels[red_labels > 0]))

    green_with_red, _ = count_overlaps(green_labels, red_bin, 0.8, True)
    purple_with_red, _ = count_overlaps(purple_labels, red_bin, 0.8, True)
    green_with_purple, _ = count_overlaps(green_labels, purple_bin, 0.8, True)
    all_markers, all_mask = count_overlaps(green_labels, (purple_bin & red_bin), 0.8, True)

    save_mask_as_zip(all_mask, str(mask_path.parent / f"{mask_path.stem}_ROI{i}_final.zip"))
    
    return {
        'Z': z_fixed,
        'ROI': f'ROI_{i+1}',
        'GFP': count_green,
        'c-fos': count_purple,
        'NeuN': count_red,
        'GFP+ NEUN+': len(green_with_red),
        'FOS+ NEUN+': len(purple_with_red),
        'GFP+ FOS+ NEUN+': len(all_markers),
        'Filename': mask_path.stem
    }


def processar_amostras(lista_amostras):
    tabela = []

    for ficheiro_czi, nome_roi, pasta_cellpose in lista_amostras:
        print(f"\n=== A processar ficheiro: {ficheiro_czi} ===")

        # Preparação
        czi = CziFile(ficheiro_czi)
        roi_zip_path = Path(nome_roi)
        rois = roiread(roi_zip_path)
        nome_base = Path(ficheiro_czi).stem
        dims = czi.get_dims_shape()[0]
        num_z = dims['Z'][1]
        px_size_x, px_size_y = get_pixel_size_from_czi(czi)
        print(f"Tamanho do píxel: {px_size_x*1e6:.3f} µm (X), {px_size_y*1e6:.3f} µm (Y)")

        # Iterar planos Z
        for z_fixo in range(num_z):
            print(f"[INFO] A processar plano Z = {z_fixo}")

            canais = ler_canais_z(Path(pasta_cellpose), z_fixo)
            if canais is None:
                print(f"[AVISO] Faltam canais para Z = {z_fixo}, a ignorar...")
                continue

            shape = canais['verde'].shape

            # Iterar ROIs
            for i, roi in enumerate(rois):
                print(f"[INFO] A processar ROI {i+1}/{len(rois)}")
                resultado = processar_roi(
                    i, roi, shape, px_size_x, px_size_y,
                    canais['verde'], canais['roxo'], canais['vermelho'],
                    z_fixo, canais['path_mascara']
                )
                resultado['Amostra'] = nome_base  # opcional: identificar a origem
                tabela.append(resultado)

    return pd.DataFrame(tabela)


In [33]:
# Step 4: Execute Analysis Pipeline
"""
This section configures and runs the complete analysis pipeline on your samples.

For each sample, you need to provide:
1. Path to the original .czi file
2. Path to the ROI file (.zip format with manually defined regions)
3. Path to the folder containing Cellpose segmentation results

The pipeline will:
- Process all Z-planes in the .czi file
- Analyze all ROIs for each Z-plane
- Count cells in each fluorescence channel
- Identify co-localized cells between channels
- Generate a comprehensive CSV report
- Export ROI files for co-localized cells
"""

# Define samples to analyze
# Each tuple contains: (czi_file_path, roi_file_path, cellpose_output_folder)
sample_list = [
    (
        "../slice_8/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08.czi",
        "../slice_8/RoiSet_brain_23_slice_8.zip",
        "../slice_8/train"
    ),
    # Add additional samples as needed:
    # ("another_file.czi", "another_roi_file.zip", "another_output_folder"),
]

# Execute the complete analysis pipeline
print("Starting cell counting and co-localization analysis...")
print("=" * 60)

df = process_samples(sample_list)

# Save results to CSV file
output_csv = 'cell_counting_results.csv'
df.to_csv(output_csv, index=False)

print(f"\n" + "=" * 60)
print("ANALYSIS COMPLETE!")
print(f"Results saved to: {output_csv}")
print("=" * 60)
print("\nSummary table:")
print(df)


=== A processar ficheiro: ../slice_8/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08.czi ===
Tamanho do píxel: 0.312 µm (X), 0.312 µm (Y)
[INFO] A processar plano Z = 0
[INFO] A processar ROI 1/2
[INFO] 5 ROIs guardadas em ..\slice_8\train\20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z0.ome_mask_ROI0_final.zip
[INFO] A processar ROI 2/2
[INFO] 8 ROIs guardadas em ..\slice_8\train\20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z0.ome_mask_ROI1_final.zip
[INFO] A processar plano Z = 1
[INFO] A processar ROI 1/2
[INFO] 24 ROIs guardadas em ..\slice_8\train\20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z1.ome_mask_ROI0_final.zip
[INFO] A processar ROI 2/2
[INFO] 13 ROIs guardadas em ..\slice_8\train\20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z1.ome_mask_ROI1_final.zip
[INFO] A processar plano Z = 2
[INFO] A processar ROI 1/2
[INFO] 5 ROIs guar